In [1]:
import os
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from functools import partial
import joblib
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression

#from src.utils import parallel_apply
#from src.feature_extraction import add_features_in_group
DIR = '/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/data'

#description = pd.read_csv(os.path.join(DIR,'HomeCredit_columns_description.csv'),encoding = 'latin1')
application = pd.read_csv(os.path.join(DIR, 'application_test.csv'))
pos_cash_balance = pd.read_csv(os.path.join(DIR, 'POS_CASH_balance.csv'))

In [2]:
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [3]:
features = pd.DataFrame({'SK_ID_CURR': pos_cash_balance['SK_ID_CURR'].unique()})

In [4]:
group_object = pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].last().reset_index()
group_object.rename(index=str,
                    columns={'CNT_INSTALMENT_FUTURE': 'pos_cash_remaining_installments'},
                    inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

,SK_ID_CURR,pos_cash_remaining_installments
0,182943,5.0
1,367990,4.0
2,397406,0.0
3,269225,24.0
4,334279,2.0


In [5]:
pos_cash_balance['is_contract_status_completed'] = pos_cash_balance['NAME_CONTRACT_STATUS'] == 'Completed'
group_object = pos_cash_balance.groupby(['SK_ID_CURR'])['is_contract_status_completed'].sum().reset_index()
group_object.rename(index=str,
                    columns={'is_contract_status_completed': 'pos_cash_completed_contracts'},
                    inplace=True)
features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

In [6]:
features

,SK_ID_CURR,pos_cash_remaining_installments,pos_cash_completed_contracts
0,182943,5.0,1
1,367990,4.0,2
2,397406,0.0,6
3,269225,24.0,5
4,334279,2.0,4
...,...,...,...
337247,185413,NaN,0
337248,326688,6.0,0
337249,394310,12.0,0
337250,240059,0.0,1


In [7]:
pos_cash_balance['pos_cash_paid_late'] = (pos_cash_balance['SK_DPD'] > 0).astype(int)
pos_cash_balance['pos_cash_paid_late_with_tolerance'] = (pos_cash_balance['SK_DPD_DEF'] > 0).astype(int)
#groupby = pos_cash_balance.groupby(['SK_ID_CURR'])

In [8]:
BUREAU_AGGREGATION_RECIPIES=[(['SK_ID_CURR'],
  [('pos_cash_paid_late', 'count'),
   ('pos_cash_paid_late', 'mean'),
   ('pos_cash_paid_late_with_tolerance', 'count'),
   ('pos_cash_paid_late_with_tolerance', 'mean'),
   ('MONTHS_BALANCE', 'mean'),
   ('MONTHS_BALANCE', 'var'),
   ('MONTHS_BALANCE', 'sum'),
   ('MONTHS_BALANCE', 'min'),
   ('SK_DPD', 'sum'),
   ('SK_DPD', 'mean'),
    ('SK_DPD', 'max'),
    ('SK_DPD', 'min'),
     ('SK_DPD', 'median'),
   ('SK_DPD_DEF', 'sum'),
   ('SK_DPD_DEF', 'mean'),
    ('SK_DPD_DEF', 'max'),
    ('SK_DPD_DEF', 'min'),
     ('SK_DPD_DEF', 'median')
   ])]

In [9]:
bureau =pos_cash_balance.sort_values(['SK_ID_CURR', 'MONTHS_BALANCE'])
#features_all = pd.DataFrame({'SK_ID_CURR': X['SK_ID_CURR'].unique()})
groupby_aggregate_names = []
number_of_applications = [1, 3, 5]
#number = 1
for number in number_of_applications:
    for groupby_cols, specs in tqdm(BUREAU_AGGREGATION_RECIPIES):
        group_object = bureau.groupby(groupby_cols).tail(number)
        group_object = group_object.groupby(groupby_cols)
        # prev_applications_tail = prev_app_sorted.groupby('SK_ID_CURR').tail(number)
        # tail_groupby = prev_applications_tail.groupby('SK_ID_CURR')
        for select, agg in tqdm(specs):
            groupby_aggregate_name = '{}_{}_{}_{}_{}'.format('_'.join(groupby_cols), agg, select,'last', number)
            features = features.merge(group_object[select]
                                .agg(agg)
                                .reset_index()
                                .rename(index=str,
                                        columns={select: groupby_aggregate_name})
                                [groupby_cols + [groupby_aggregate_name]],
                                on=groupby_cols,
                                how='left')
            groupby_aggregate_names.append(groupby_aggregate_name)

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_86803/213735849.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for groupby_cols, specs in tqdm(BUREAU_AGGREGATION_RECIPIES):


  0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_86803/213735849.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for select, agg in tqdm(specs):


  0%|          | 0/18 [00:00<?, ?it/s]

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_86803/213735849.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for groupby_cols, specs in tqdm(BUREAU_AGGREGATION_RECIPIES):


...

  0%|          | 0/18 [00:00<?, ?it/s]

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_86803/213735849.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for groupby_cols, specs in tqdm(BUREAU_AGGREGATION_RECIPIES):


  0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_86803/213735849.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for select, agg in tqdm(specs):


  0%|          | 0/18 [00:00<?, ?it/s]

In [10]:
features

,SK_ID_CURR,pos_cash_remaining_installments,pos_cash_completed_contracts,SK_ID_CURR_count_pos_cash_paid_late_last_1,SK_ID_CURR_mean_pos_cash_paid_late_last_1,SK_ID_CURR_count_pos_cash_paid_late_with_tolerance_last_1,SK_ID_CURR_mean_pos_cash_paid_late_with_tolerance_last_1,SK_ID_CURR_mean_MONTHS_BALANCE_last_1,SK_ID_CURR_var_MONTHS_BALANCE_last_1,SK_ID_CURR_sum_MONTHS_BALANCE_last_1,...,SK_ID_CURR_sum_SK_DPD_last_5,SK_ID_CURR_mean_SK_DPD_last_5,SK_ID_CURR_max_SK_DPD_last_5,SK_ID_CURR_min_SK_DPD_last_5,SK_ID_CURR_median_SK_DPD_last_5,SK_ID_CURR_sum_SK_DPD_DEF_last_5,SK_ID_CURR_mean_SK_DPD_DEF_last_5,SK_ID_CURR_max_SK_DPD_DEF_last_5,SK_ID_CURR_min_SK_DPD_DEF_last_5,SK_ID_CURR_median_SK_DPD_DEF_last_5
0,182943,5.0,1,1,0.0,1,0.0,-2.0,NaN,-2,...,0,0.0,0,0,0.0,0,0.0,0,0,0.0
1,367990,4.0,2,1,0.0,1,0.0,-16.0,NaN,-16,...,0,0.0,0,0,0.0,0,0.0,0,0,0.0
2,397406,0.0,6,1,0.0,1,0.0,-1.0,NaN,-1,...,0,0.0,0,0,0.0,0,0.0,0,0,0.0
3,269225,24.0,5,1,0.0,1,0.0,-1.0,NaN,-1,...,0,0.0,0,0,0.0,0,0.0,0,0,0.0
4,334279,2.0,4,1,0.0,1,0.0,-1.0,NaN,-1,...,0,0.0,0,0,0.0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337247,185413,NaN,0,1,0.0,1,0.0,-2.0,NaN,-2,...,0,0.0,0,0,0.0,0,0.0,0,0,0.0
337248,326688,6.0,0,1,0.0,1,0.0,-2.0,NaN,-2,...,0,0.0,0,0,0.0,0,0.0,0,0,0.0
337249,394310,12.0,0,1,0.0,1,0.0,-1.0,NaN,-1,...,0,0.0,0,0,0.0,0,0.0,0,0,0.0
337250,240059,0.0,1,1,0.0,1,0.0,-1.0,NaN,-1,...,0,0.0,0,0,0.0,0,0.0,0,0,0.0


In [11]:
features.to_parquet('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/features_test/pos_cash_features.parquet')